In [ ]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
import os
from dotenv import load_dotenv
from typing import Optional
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
load_dotenv()
import json
import requests
import pandas as pd

In [ ]:
import pandas as pd

def process_excel_to_json(excel_file_path):
    """
    Reads data from an Excel file, processes it, and returns the data as a JSON string.

    Parameters:
    excel_file_path (str): The path to the Excel file.

    Returns:
    str: JSON string representing the processed data.
    """
    # Read data from the Excel file into a pandas DataFrame
    df = pd.read_excel(excel_file_path)

    # Process the data: drop NaN values in each row and convert to JSON
    json_str = df.apply(lambda x: [x.dropna()], axis=1).to_json(orient='index', date_format='iso')

    # Return the JSON string
    return json_str

# Example usage:
# Provide the path to your Excel file
excel_file_path = 'data\\AI water oncharging example3.xlsx'
processed_json = process_excel_to_json(excel_file_path)
print(processed_json)

In [ ]:
# synthetic data for testing
synthetic_data = {'id': '123456',
 'name_of_the_company': 'crayon Pty Ltd',
 'address': '1788 dandenong road clayton-3168',
 'issued_to': 'Maximum life PVT LTD',
 'total_amount': '$1000',
 'balance': '$1000',
 'invoice_number': '8540',
 'invoice_date': '31/09/2022',
 'shipment_details': '',
 'payment_terms': 'Net 40'}

In [ ]:
example_json ={
   "title":"id",
   "description":"Extract information from invoice",
   "type":"object",
   "properties":{
      "Vendor No.":{
         "title":"Vendor No.",
         "description":"Identify the ID of the vendor",
         "type":"string"
      },
      "Vendor Name":{
         "title":"Vendor Name",
         "description":"Identify the name of the vendor",
         "type":"string"
      },
      "Account No.":{
         "title":"Account No.",
         "description":"Identify the account number",
         "type":"string"
      },
      "Property Address":{
         "title":"Property Address",
         "description":"Identify the Property Address",
         "type":"string"
      },
      "Posted Purchase Invoice":{
         "title":"Posted Purchase Invoice",
         "description":"Identify the Posted Purchase Invoice",
         "type":"string"
      },
      "Invoice Date":{
         "title":"Invoice Date",
         "description":"Identify the Invoice Date",
         "type":"string"
      },
      "Fixed Charge Start Date":{
         "title":"Fixed Charge Start Date",
         "description":"Identify the Fixed Charge Start Date",
         "type":"string"
      },
      "Fixed Charge End Date":{
         "title":"Fixed Charge End Date",
         "description":"Identify the Fixed Charge End Date",
         "type":"string"
      },
      "Fixed Charge for Water Service":{
         "title":"Fixed Charge for Water Service",
         "description":"Identify the Fixed Charge for Water Service",
         "type":"string"
      },
      "Fixed Charge for Waste Water":{
         "title":"Fixed Charge for Waste Water",
         "description":"Identify the Fixed Charge for Waste Water",
         "type":"string"
      },
      "Total Fixed Charges":{
         "title":"Total Fixed Charges",
         "description":"Identify the Total Fixed Charges",
         "type":"string"
      },
      "line_items":{
         "title":"line items",
         "description":"What are line items such as Water Oncharging mentioned? Please put them in the following JSON format.",
         "type":"array",
         "items":{
            "title":"line item",
            "description":"Extract information from each line Water Oncharging",
            "type":"object",
            "properties":{
               "Reading From (Date)":{
                  "title":"Reading From (Date)",
                  "description":"Identify the Reading From (Date)",
                  "type":"string"
               },
               "Reading To (Date)":{
                  "title":"Reading To (Date)",
                  "description":"Identify the Reading To (Date)",
                  "type":"string"
               },
               "No. of Days":{
                  "title":"No. of Days",
                  "description":"Identify the number of Days",
                  "type":"string"
               },
               "Meter No.":{
                  "title":"Meter No.",
                  "description":"Identify the Meter No.",
                  "type":"string"
               },
               "Last Reading":{
                  "title":"Last Reading",
                  "description":"Identify the Last Reading",
                  "type":"string"
               },
               "This Reading":{
                  "title":"This Reading",
                  "description":"Identify the This Reading",
                  "type":"string"
               },
               "Net Meter Reading":{
                  "title":"Net Meter Reading",
                  "description":"Identify the Net Meter Reading",
                  "type":"string"
               },
               "Rate 1":{
                  "title":"Rate 1",
                  "description":"Identify the Rate 1",
                  "type":"string"
               },
               "Usage 1":{
                  "title":"Usage 1",
                  "description":"Identify the Usage 1",
                  "type":"string"
               },
               "Usage Cost 1":{
                  "title":"Usage Cost 1",
                  "description":"Identify the Usage Cost 1",
                  "type":"string"
               },
               "Rate 2":{
                  "title":"Rate 2",
                  "description":"Identify the Rate 2",
                  "type":"string"
               },
               "Usage 2":{
                  "title":"Usage 2",
                  "description":"Identify the Usage 2",
                  "type":"string"
               },
               "Usage Cost 2":{
                  "title":"Usage Cost 2",
                  "description":"Identify the Usage Cost 2",
                  "type":"string"
               },
               "Rate 3":{
                  "title":"Rate 3",
                  "description":"Identify the Rate 3",
                  "type":"string"
               },
               "Usage 3":{
                  "title":"Usage 3",
                  "description":"Identify the Usage 3",
                  "type":"string"
               },
               "Usage Cost 3":{
                  "title":"Usage Cost 3",
                  "description":"Identify the Usage Cost 3",
                  "type":"string"
               },
               "Total Usage":{
                  "title":"Total Usage",
                  "description":"Identify the Total Usage",
                  "type":"string"
               },
               "Other Water Charges":{
                  "title":"Other Water Charges",
                  "description":"Identify the Other Water Charges",
                  "type":"string"
               },
               "Other Water Credits":{
                  "title":"Other Water Credits",
                  "description":"Identify the Other Water Credits",
                  "type":"string"
               },
               "Total Charge":{
                  "title":"Total Charge",
                  "description":"Identify the Total Charge",
                  "type":"string"
               }
            },
            "required":[
               "Reading From (Date)",
               "Total Charge"
            ]
         }
      }
   },
   "required":[
      "id",
      "Vendor Name",
      "Property Address",
      "line_items"
   ]
}

In [ ]:
template = '''
System: You are an AI assistant. Please follow the below example and extract the json Schema for the input json given 


Example 1

json instance: {output}
jSON Schema: {json_schema}

json instance:{input}
jSON Schema:

'''

In [ ]:
def generate_schema_response(template,final_output, json_str, example_json):
    """
    Generates a chat response using the specified template and input data.

    Parameters:
    template (str): Template for formatting the chat prompt.
    json_str (str): JSON string data.
    raw_content (str): Raw content from the document.
    json_schema (str): JSON schema information.

    Returns:
    str: Chat response generated by GPT-4 model.
    """
    gpt3_5_prompt = PromptTemplate(
        template=template,
        input_variables=['output', 'input', 'json_schema']
    )

    system_message_prompt = SystemMessagePromptTemplate(prompt=gpt3_5_prompt)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

    # Format the chat prompt using the provided input data
    messages = chat_prompt.format_prompt(output=final_output, input=json_str , json_schema=example_json).to_messages()

    # Create a GPT-4 chat instance
    chat = ChatOpenAI(engine="gpt-4-32k", temperature=0)

    # Generate the chat response using the formatted messages
    resp = chat(messages)

    # Return the content of the response
    return resp.content

In [ ]:
response = generate_schema_response(template,final_output, processed_json, example_json)